In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
batch_size = 128
num_epochs = 200
learning_rate = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 數據增強 & 載入 CIFAR-100
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),   # 隨機裁切
    transforms.RandomHorizontalFlip(),      # 隨機翻轉
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761)) # CIFAR-100 mean/std
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),
                         (0.2675, 0.2565, 0.2761))
])

trainset = torchvision.datasets.CIFAR100(root="./data", train=True,
                                         download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root="./data", train=False,
                                        download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


100%|██████████| 169M/169M [00:39<00:00, 4.29MB/s] 


In [ ]:
from torchvision.models import resnet18

model = resnet18(weights=None)  # 不載入 ImageNet 預訓練權重
model.fc = nn.Linear(model.fc.in_features, 100)  # CIFAR-100 = 100 類別
model = model.to(device)

# 定義 Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.2)  # 學習率衰減

In [ ]:
# 訓練 & 測試函式
def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in trainloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {running_loss/len(trainloader):.3f} | "
          f"Train Acc: {100.*correct/total:.2f}%")

def test(epoch):
    model.eval()
    loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total
    print(f"Test Loss: {loss/len(testloader):.3f} | Test Acc: {acc:.2f}%")
    return acc

In [ ]:
# 主迴圈
best_acc = 0
for epoch in range(num_epochs):
    train(epoch)
    acc = test(epoch)
    scheduler.step()

    # 儲存最佳模型
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "resnet18_cifar100.pth")
        print("✅ Saved Best Model!")

print(f"\n🎉 Training Finished! Best Test Acc: {best_acc:.2f}%")


'''
Epoch [158/200] Train Loss: 0.027 | Train Acc: 99.63%
Test Loss: 1.836 | Test Acc: 60.88%
'''

Epoch [1/200] Train Loss: 3.999 | Train Acc: 9.53%
Test Loss: 3.520 | Test Acc: 15.54%
✅ Saved Best Model!
Epoch [2/200] Train Loss: 3.370 | Train Acc: 17.84%
Test Loss: 3.203 | Test Acc: 21.41%
✅ Saved Best Model!
Epoch [3/200] Train Loss: 3.079 | Train Acc: 23.24%
Test Loss: 2.915 | Test Acc: 26.66%
✅ Saved Best Model!
Epoch [4/200] Train Loss: 2.850 | Train Acc: 27.50%
Test Loss: 2.693 | Test Acc: 31.30%
✅ Saved Best Model!
Epoch [5/200] Train Loss: 2.698 | Train Acc: 30.71%
Test Loss: 2.697 | Test Acc: 30.60%
Epoch [6/200] Train Loss: 2.578 | Train Acc: 33.32%
Test Loss: 2.477 | Test Acc: 35.13%
✅ Saved Best Model!
Epoch [7/200] Train Loss: 2.491 | Train Acc: 35.07%
Test Loss: 2.477 | Test Acc: 36.04%
✅ Saved Best Model!
Epoch [8/200] Train Loss: 2.413 | Train Acc: 36.66%
Test Loss: 2.476 | Test Acc: 35.96%
Epoch [9/200] Train Loss: 2.354 | Train Acc: 37.92%
Test Loss: 2.427 | Test Acc: 36.88%
✅ Saved Best Model!
Epoch [10/200] Train Loss: 2.289 | Train Acc: 39.32%
Test Loss: 2.360

KeyboardInterrupt: 